In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer

# 1 Часть (сделана полностью)

In [3]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [5]:
data_feat = data.drop('MEDV',axis=1)
data_targ = data['MEDV']
x_train, x_test, y_train, y_test = train_test_split(data_feat, 
                                                    data_targ, 
                                                    test_size=0.2, 
                                                    random_state=RANDOM_STATE 
                                                    )

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
%%time
# Линейная регрессия
LR = LinearRegression()
LR.fit(x_train, y_train)

y_pred = LR.predict(x_test)
r2_lr = r2_score(y_test, y_pred)

# Ridge

Ridge = Ridge()
Ridge.fit(x_train, y_train)

y_pred_r = Ridge.predict(x_test)
r2_ridge = r2_score(y_test, y_pred_r)

# Lasso

Lasso = Lasso()
Lasso.fit(x_train, y_train)

y_pred_l = Lasso.predict(x_test)
r2_lasso = r2_score(y_test, y_pred_l)

print('r2_lr:', r2_lr)
print('r2_ridge:', r2_ridge)
print('r2_lasso:', r2_lasso)

# r2_lr = 0
# r2_ridge = 0
# r2_lasso = 0
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

r2_lr: 0.6684825753971606
r2_ridge: 0.6659608075261692
r2_lasso: 0.6668687223368214
CPU times: total: 109 ms
Wall time: 253 ms


In [7]:
print(f'Лучший результат у линейной регрессии: R2 = {r2_lr}')

Лучший результат у линейной регрессии: R2 = 0.6684825753971606


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [8]:
parametrs = {'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}

# Ridge_GridSearchCV

Ridge_GS = GridSearchCV(Ridge, parametrs)
Ridge_GS.fit(x_train, y_train)
y_pred_gs_r = Ridge_GS.predict(x_test)
r2_ridge_grid_search = r2_score(y_test, y_pred_gs_r)
print('r2_ridge_grid_search:', r2_ridge_grid_search)

# RidgeCV

Ridge_CV = RidgeCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]).fit(data_feat, data_targ)
y_pred_r_cv = Ridge_CV .predict(x_test)
r2_ridge_cv = r2_score(y_test, y_pred_r_cv)
print('r2_ridge_cv:', r2_ridge_cv)

# Lasso_GridSearchCV

Lasso_GS = GridSearchCV(Lasso, parametrs)
Lasso_GS.fit(x_train, y_train)
y_pred_gs_l = Lasso_GS.predict(x_test)
r2_lasso_grid_search = r2_score(y_test, y_pred_gs_l)
print('r2_lasso_grid_search:', r2_lasso_grid_search)

# LassoCV

Lasso_CV = LassoCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], random_state=RANDOM_STATE).fit(data_feat, data_targ)
y_pred_l_cv = Lasso_CV.predict(x_test)
r2_lasso_cv = r2_score(y_test, y_pred_l_cv)
print('r2_lasso_cv:', r2_lasso_cv)

# r2_ridge_grid_search = 0
# r2_ridge_cv = 0
# r2_lasso_grid_search = 0
# r2_lasso_cv = 0
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

r2_ridge_grid_search: 0.6684825680074256
r2_ridge_cv: 0.7024461116760787
r2_lasso_grid_search: 0.6684829595885677
r2_lasso_cv: 0.6883086259341868


In [9]:
print(f'Лучший результат у "RidgeCV": R2 = {r2_ridge_cv}. По сравнению с лучшим результатом на предыдущем этапе R2 улучшился на {r2_ridge_cv - r2_lr}"')

Лучший результат у "RidgeCV": R2 = 0.7024461116760787. По сравнению с лучшим результатом на предыдущем этапе R2 улучшился на 0.03396353627891813"


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [12]:
# StandardScaler подготовка данных

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# MinMaxScaler подготовка данных

min_max_scaler = MinMaxScaler()
x_train_scaled_m = min_max_scaler.fit_transform(x_train)
x_test_scaled_m = min_max_scaler.transform(x_test)

#Ridge StandardScaler

Ridge_1 = Ridge()
Ridge_1.fit(x_train_scaled, y_train)

y_pred_r_1 = Ridge_1.predict(x_test_scaled)
r2_ridge_standart_scaler = r2_score(y_test, y_pred_r_1)
print('r2_ridge_standart_scaler:', r2_ridge_standart_scaler)

#Lasso StandardScaler

Lasso_1 = Lasso()
Lasso_1.fit(x_train_scaled, y_train)

y_pred_l_1 = Lasso_1.predict(x_test_scaled)
r2_lasso_standart_scaler = r2_score(y_test, y_pred_l_1)
print('r2_lasso_standart_scaler:', r2_lasso_standart_scaler)

#Ridge MinMaxScaler

Ridge_MinMaxScaler = Ridge()
Ridge_MinMaxScaler.fit(x_train_scaled_m, y_train)

y_pred_r_MinMaxScaler = Ridge_MinMaxScaler.predict(x_test_scaled_m)
r2_ridge_min_max_scaler = r2_score(y_test, y_pred_r_MinMaxScaler)
print('r2_ridge_min_max_scaler:', r2_ridge_min_max_scaler)

#Lasso MinMaxScaler

Lasso_MinMaxScaler = Lasso()
Lasso_MinMaxScaler.fit(x_train_scaled_m, y_train)

y_pred_l_MinMaxScaler = Lasso_MinMaxScaler.predict(x_test_scaled_m)
r2_lasso_min_max_scaler = r2_score(y_test, y_pred_l_MinMaxScaler)
print('r2_lasso_min_max_scaler:', r2_lasso_min_max_scaler)

# r2_ridge_standart_scaler = 0
# r2_ridge_min_max_scaler = 0
# r2_lasso_standart_scaler = 0
# r2_lasso_min_max_scaler = 0
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

r2_ridge_standart_scaler: 0.6681901076774426
r2_lasso_standart_scaler: 0.6240447523478461
r2_ridge_min_max_scaler: 0.6762207658974593
r2_lasso_min_max_scaler: 0.2573921442545194


In [13]:
print(f'Лучший результат на данном этапе у "Lasso_StandardScaler": R2 = {r2_lasso_standart_scaler}') 
print(f'По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = {r2_ridge_cv}) R2 ухудшился на {r2_ridge_cv - r2_lasso_standart_scaler}')

Лучший результат на данном этапе у "Lasso_StandardScaler": R2 = 0.6240447523478461
По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = 0.7024461116760787) R2 ухудшился на 0.07840135932823267


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [14]:
# StandardScaler подготовка данных

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# MinMaxScaler подготовка данных

min_max_scaler = MinMaxScaler()
x_train_scaled_m = min_max_scaler.fit_transform(x_train)
x_test_scaled_m = min_max_scaler.transform(x_test)

# Ridge_StandardScaler_CV

R_SS_CV = RidgeCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]).fit(x_train_scaled, y_train)
y_pred_r_ss_cv = R_SS_CV.predict(x_test_scaled)
r2_ridge_standart_scaler_cv = r2_score(y_test, y_pred_r_ss_cv)
print('r2_ridge_standart_scaler_cv:', r2_ridge_standart_scaler_cv)

# Ridge_MinMaxScaler_CV

R_MM_CV = RidgeCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]).fit(x_train_scaled_m, y_train)
y_pred_r_mm_cv = R_MM_CV.predict(x_test_scaled_m)
r2_ridge_min_max_scaler_cv = r2_score(y_test, y_pred_r_mm_cv)
print('r2_ridge_min_max_scaler_cv:', r2_ridge_min_max_scaler_cv)

# Lasso_StandardScaler_CV

L_SS_CV = LassoCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], random_state=RANDOM_STATE).fit(x_train_scaled, y_train)
y_pred_l_ss_cv = L_SS_CV.predict(x_test_scaled)
r2_lasso_standart_scaler_cv = r2_score(y_test, y_pred_l_ss_cv)
print('r2_lasso_standart_scaler_cv:', r2_lasso_standart_scaler_cv)

# Lasso_MinMaxScaler_CV

L_MM_CV = LassoCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], random_state=RANDOM_STATE).fit(x_train_scaled_m, y_train)
y_pred_l_mm_cv = L_MM_CV.predict(x_test_scaled_m)
r2_lasso_min_max_scaler_cv = r2_score(y_test, y_pred_l_mm_cv)
print('r2_lasso_min_max_scaler_cv:', r2_lasso_min_max_scaler_cv)

# r2_ridge_standart_scaler_cv = 0
# r2_ridge_min_max_scaler_cv = 0
# r2_lasso_standart_scaler_cv = 0
# r2_lasso_min_max_scaler_cv = 0
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

r2_ridge_standart_scaler_cv: 0.6657305340616932
r2_ridge_min_max_scaler_cv: 0.6697651112860057
r2_lasso_standart_scaler_cv: 0.6684821312777707
r2_lasso_min_max_scaler_cv: 0.668483656442819


In [15]:
print(f'Лучший результат на данном этапе у "Ridge_MinMaxScaler_CV": R2 = {r2_ridge_min_max_scaler_cv}') 
print(f'По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = {r2_ridge_cv}) R2 ухудшился на {r2_ridge_cv - r2_ridge_min_max_scaler_cv}')

Лучший результат на данном этапе у "Ridge_MinMaxScaler_CV": R2 = 0.6697651112860057
По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = 0.7024461116760787) R2 ухудшился на 0.032681000390073045


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [16]:
# StandardScaler подготовка данных

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# MinMaxScaler подготовка данных

min_max_scaler = MinMaxScaler()
x_train_scaled_m = min_max_scaler.fit_transform(x_train)
x_test_scaled_m = min_max_scaler.transform(x_test)

# Прогоняем через Polynomial

poly = PolynomialFeatures(2)

x_train_sscaled_poly = poly.fit_transform(x_train_scaled)
x_test_sscaled_poly = poly.fit_transform(x_test_scaled)

x_train_sscaled_m_poly = poly.fit_transform(x_train_scaled_m)
x_test_sscaled_m_poly = poly.fit_transform(x_test_scaled_m)

# r2_ridge_standart_scaler_poly

Ridge_2 = Ridge()
Ridge_2.fit(x_train_sscaled_poly, y_train)

y_pred_r_2 = Ridge_2.predict(x_test_sscaled_poly)
r2_ridge_standart_scaler_poly = r2_score(y_test, y_pred_r_2)
print('r2_ridge_standart_scaler_poly:', r2_ridge_standart_scaler_poly)

# r2_ridge_min_max_scaler_poly

Ridge_MinMaxScaler_2 = Ridge()
Ridge_MinMaxScaler_2.fit(x_train_sscaled_m_poly, y_train)

y_pred_r_MinMaxScaler_2 = Ridge_MinMaxScaler_2.predict(x_test_sscaled_m_poly)
r2_ridge_min_max_scaler_poly = r2_score(y_test, y_pred_r_MinMaxScaler_2)
print('r2_ridge_min_max_scaler_poly:', r2_ridge_min_max_scaler_poly)

# r2_lasso_standart_scaler_poly

Lasso_3 = Lasso()
Lasso_3.fit(x_train_sscaled_poly, y_train)

y_pred_l_3 = Lasso_3.predict(x_test_sscaled_poly)
r2_lasso_standart_scaler_poly = r2_score(y_test, y_pred_l_3)
print('r2_lasso_standart_scaler_poly:', r2_lasso_standart_scaler_poly)

# r2_lasso_min_max_scaler_poly

Lasso_MinMaxScaler_3 = Lasso()
Lasso_MinMaxScaler_3.fit(x_train_sscaled_m_poly, y_train)

y_pred_l_MinMaxScaler_3 = Lasso_MinMaxScaler_3.predict(x_test_sscaled_m_poly)
r2_lasso_min_max_scaler_poly = r2_score(y_test, y_pred_l_MinMaxScaler_3)
print('r2_lasso_min_max_scaler_poly:', r2_lasso_min_max_scaler_poly)

# r2_ridge_standart_scaler_poly = 0
# r2_ridge_min_max_scaler_poly = 0
# r2_lasso_standart_scaler_poly = 0
# r2_lasso_min_max_scaler_poly = 0
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

r2_ridge_standart_scaler_poly: 0.8171359789657142
r2_ridge_min_max_scaler_poly: 0.8298862467502752
r2_lasso_standart_scaler_poly: 0.7322738282708989
r2_lasso_min_max_scaler_poly: 0.26112627417356593


In [17]:
print(f'Лучший результат на данном этапе у "Ridge_MinMaxScaler_Poly": R2 = {r2_ridge_min_max_scaler_poly}') 
print(f'По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = {r2_ridge_cv}) R2 улучшился на {r2_ridge_min_max_scaler_poly - r2_ridge_cv}')

Лучший результат на данном этапе у "Ridge_MinMaxScaler_Poly": R2 = 0.8298862467502752
По сравнению с лучшим результатом на предыдущих этапах ("RidgeCV" R2 = 0.7024461116760787) R2 улучшился на 0.1274401350741965


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [18]:
# Прогоняем через Polynomial

poly = PolynomialFeatures(2)

x_train_sscaled_poly = poly.fit_transform(x_train_scaled)
x_test_sscaled_poly = poly.fit_transform(x_test_scaled)

x_train_sscaled_m_poly = poly.fit_transform(x_train_scaled_m)
x_test_sscaled_m_poly = poly.fit_transform(x_test_scaled_m)

# r2_ridge_standart_scaler_poly_cv
R_SS_CV_poly = RidgeCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]).fit(x_train_sscaled_poly, y_train)
y_pred_r_ss_cv_poly = R_SS_CV_poly.predict(x_test_sscaled_poly)
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, y_pred_r_ss_cv_poly)

# r2_ridge_min_max_scaler_poly_cv
R_MM_CV_poly = RidgeCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]).fit(x_train_sscaled_m_poly, y_train)
y_pred_r_mm_cv_poly = R_MM_CV_poly.predict(x_test_sscaled_m_poly)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, y_pred_r_mm_cv_poly)

# r2_lasso_standart_scaler_poly_cv
L_SS_CV_poly = LassoCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], random_state=RANDOM_STATE).fit(x_train_sscaled_poly, y_train)
y_pred_l_ss_cv_poly = L_SS_CV_poly.predict(x_test_sscaled_poly)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, y_pred_l_ss_cv_poly)

# r2_lasso_min_max_scaler_poly_cv
L_MM_CV_poly = LassoCV(alphas=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], random_state=RANDOM_STATE).fit(x_train_sscaled_m_poly, y_train)
y_pred_l_mm_cv_poly = L_MM_CV_poly.predict(x_test_sscaled_m_poly)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, y_pred_l_mm_cv_poly)

print('r2_ridge_standart_scaler_poly_cv:', r2_ridge_standart_scaler_poly_cv)
print('r2_ridge_min_max_scaler_poly_cv:', r2_ridge_min_max_scaler_poly_cv)
print('r2_lasso_standart_scaler_poly_cv:', r2_lasso_standart_scaler_poly_cv)
print('r2_lasso_min_max_scaler_poly_cv:', r2_lasso_min_max_scaler_poly_cv)


# r2_ridge_standart_scaler_poly_cv = 0
# r2_ridge_min_max_scaler_poly_cv = 0
# r2_lasso_standart_scaler_poly_cv = 0
# r2_lasso_min_max_scaler_poly_cv = 0
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.7088881028601, tolerance: 2.882109046439629
  model = cd_fast.enet_coordinate_descent_gram(
c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 325.3702094707344, tolerance: 2.882109046439629
  model = cd_fast.enet_coordinate_descent_gram(
c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 649.7148073796305, tolerance: 2.882109046439629
  model = cd_fast.enet_coordinate_descent_gram(
c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You 

r2_ridge_standart_scaler_poly_cv: 0.8187344606117395
r2_ridge_min_max_scaler_poly_cv: 0.8501402125962156
r2_lasso_standart_scaler_poly_cv: 0.8128138856150267
r2_lasso_min_max_scaler_poly_cv: 0.8397022991682292


In [19]:
print(f'Лучший результат на данном этапе у "Ridge_MinMaxScaler_Poly_CV": R2 = {r2_ridge_min_max_scaler_poly_cv}') 
print(f'По сравнению с лучшим результатом на предыдущих этапах ("Ridge_MinMaxScaler_Poly" R2 = {r2_ridge_min_max_scaler_poly}) R2 улучшился на {r2_ridge_min_max_scaler_poly_cv - r2_ridge_min_max_scaler_poly}')

Лучший результат на данном этапе у "Ridge_MinMaxScaler_Poly_CV": R2 = 0.8501402125962156
По сравнению с лучшим результатом на предыдущих этапах ("Ridge_MinMaxScaler_Poly" R2 = 0.8298862467502752) R2 улучшился на 0.020253965845940414


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [20]:
# Я так понял, лучшую модель мы выбирали в задачах выше. Чем ближе R2 к единице, тем лучше модель.

# best_params = {}
# print('Параметры лучшей модели:\n', best_params)
# r2_best_model = 0
# results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

results_regression.sort_values(by='R2', ascending = False).head(1)

,model,task,R2
20,Ridge_MinMaxScaler_Poly_CV,task7,0.85014


Вывод: лучшая модель - Ridge_MinMaxScaler_Poly_CV

In [21]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.702446
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.688309
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


# 2 Часть (сделана не полностью)

http://archive.ics.uci.edu/ml/datasets/Adult

In [22]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [23]:
data.loc[data["class"] == "<=50K", "class"] = 1
data.loc[data["class"] == ">50K", "class"] = 0
data["class"] = pd.to_numeric(data["class"])
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [24]:
# data_features = data.drop('class',axis=1)
# data_target = data['class']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [25]:
print(data['class'].value_counts())

class
1    37155
0    11687
Name: count, dtype: int64


### Расчет метрик
Всего = 48 842

Смоделируем:
Класс 1 = 37 155
Класс 0 = 11 687

Предположим, что класс 1 это все верные ответы, а класс 0 это все не найденные обьекты моделью ответы.

Если мы считаем только по самому частому классу, то получится:
f1 мера

precision
 TP = будем считать, что это 36 155 (действительное количество правильных ответов)
 FP = будем считать, что это 2 000 (это значение возьмем от класса 0 и предположим, что модель ошиблась, назвав этот набор
 данных тоже классом 1)

recall
 FN = будем считать, что это 1 000 (это значение возьмем от класса 1 и предположим, что модель ошиблась, назвав этот набор
 данных классом 0)

In [26]:
acc_most_frequent = 37155/48842
print('acc_most_frequent:', acc_most_frequent)
print('----')
precision = 36155 / (36155+2000)
print('precision:', precision)
print('----')
recall = 36155 / (36155+1000)
print('recall:', recall)
print('----')
f1_most_frequent = 2 * (0.94 * 0.97) / (0.94 + 0.97)
print('f1_most_frequent:', f1_most_frequent)

# f1_most_frequent = 0
# acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

acc_most_frequent: 0.7607182343065395
----
precision: 0.9475822303760975
----
recall: 0.973085721975508
----
f1_most_frequent: 0.9547643979057592


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [27]:
data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

Пропуски в данных отсутствуют

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [28]:
# tasks+=1

data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
class              int64
dtype: object

age, fnlwgt, education-num, capital-gain, capital-loss, hours-per-week, class - числовые

workclass, education, marital-status, occupation, relationship, race, sex, native-country - категориальные 

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [1]:
f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

NameError: name 'LogisticRegression' is not defined

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification